### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ

#### Факультет физико-математических и естественных наук  
#### Кафедра математического моделирования и искусственного интеллекта

## ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2

#### Дисциплина: Интеллектуальный анализ данных

##### Студент:  Стелина Петрити
##### Группа:   НПИбд-02-21

## Москва 2024
***

### Вариант № 9
1. Алгоритм: FPGrowth
2. День недели (поле order_dow таблицы orders): “4”
3. Код департамента (поле department_id таблицы products): “10”
4. Запрос: определить час дня, в который было приобретено меньше всего различных товаров
5. Показатель оценки ассоциативных правил: рычаг (leverage)


### 1.	При помощи модуля sqlite3 откройте базу данных Instacart в файле inst

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sqlite3
path = '/content/drive/MyDrive/viti 4/Data mining/Lab 2/instacart.db'

# Подключение к базе данных
conn = sqlite3.connect(path)
cursor = conn.cursor()

# Пример запроса для проверки работоспособности соединения
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
# Распечатать таблицы в базе данных
print(tables)
conn.close()


[('aisles',), ('products',), ('departments',), ('orders',), ('order_products__train',)]


### 2. Загрузите таблицы departments и products в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы order_products__train записи, соответствующие указанным в индивидуальном задании дню недели (поле order_dow таблицы orders) и коду департамента (поле department_id таблицы products) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме, количество транзакций (покупок) и определите количество товаров (столбец product_id) в транзакциях датафрейма.

In [ ]:
import pandas as pd

# Подключаемся к базе данных
conn = sqlite3.connect(path)

# Замените 'table_name' на имя вашей таблицы
departments_path = pd.read_sql_query("SELECT * FROM departments", conn)
products_path = pd.read_sql_query("SELECT * FROM products", conn)

# Проверяем первые строки каждого датафрейма
print(departments_path.head())
print(products_path.head())


  department_id department
0             1     frozen
1             2      other
2             3     bakery
3             4    produce
4             5    alcohol
  product_id                                       product_name aisle_id  \
0          1                         Chocolate Sandwich Cookies       61   
1          2                                   All-Seasons Salt      104   
2          3               Robust Golden Unsweetened Oolong Tea       94   
3          4  Smart Ones Classic Favorites Mini Rigatoni Wit...       38   
4          5                          Green Chile Anytime Sauce        5   

  department_id  
0            19  
1            13  
2             7  
3             1  
4            13  


In [ ]:
# День недели (четверг) и код департамента (10)
order_dow = 4  # Четверг
department_id = 10  # Код департамента

# SQL-запрос
query = """
SELECT opt.order_id, opt.product_id, opt.add_to_cart_order, opt.reordered
FROM order_products__train opt
JOIN orders o ON opt.order_id = o.order_id
JOIN products p ON opt.product_id = p.product_id
WHERE o.order_dow = ? AND p.department_id = ?
"""

# Выполняем запрос и загружаем данные в DataFrame
df = pd.read_sql_query(query, conn, params=(order_dow, department_id))

# Проверяем первые строки датафрейма
print(df.head())



  order_id product_id add_to_cart_order reordered
0  2560500       1000                18         1
1  1229133      22260                 9         1
2    70373       5161                11         0
3  2082798       5161                 9         1
4  1849852      10915                 1         1


In [ ]:
# Количество строк в датафрейме (всего записей)
number_of_rows = df.shape[0]

# Количество уникальных транзакций (по количеству уникальных значений order_id)
number_of_transactions = df['order_id'].nunique()

# Количество уникальных продуктов (по количеству уникальных значений product_id)
number_of_products = df['product_id'].nunique()

# Выводим результаты
print(f"Количество строк в датафрейме: {number_of_rows}")
print(f"Количество уникальных транзакций (покупок): {number_of_transactions}")
print(f"Количество уникальных товаров: {number_of_products}")


Количество строк в датафрейме: 161
Количество уникальных транзакций (покупок): 155
Количество уникальных товаров: 25


### 3. Выполните к датафрейму запрос, указанный в индивидуальном задании.

In [ ]:
conn = sqlite3.connect(path)
# SQL-запрос для извлечения данных
query = """
SELECT orders.order_hour_of_day, order_products__train.product_id
FROM order_products__train
JOIN orders ON order_products__train.order_id = orders.order_id
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Группировка по часу дня и подсчёт уникальных товаров
hourly_products = df.groupby('order_hour_of_day')['product_id'].nunique()
minimum_hour = hourly_products.idxmin()
min_unique_products = hourly_products.min()

print(f"Час дня с наименьшим количеством уникальных товаров: {minimum_hour}")
print(f"Количество уникальных товаров в этот час: {min_unique_products}")


Час дня с наименьшим количеством уникальных товаров: 03
Количество уникальных товаров в этот час: 1735


###4.	Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец order_id, а в качестве названий товаров - поле product_name из датафрейма для таблицы products, соответствующее столбцу product_id. Найдите в транзакционной базе данных транзакцию с наибольшим количеством товаров и выведите ее на экран.

In [ ]:
# Подключаемся к базе данных
conn = sqlite3.connect(path)

# Загружаем таблицы в датафреймы
orders_df = pd.read_sql_query("SELECT * FROM orders", conn)
products_df = pd.read_sql_query("SELECT * FROM products", conn)
order_products_train_df = pd.read_sql_query("SELECT * FROM order_products__train", conn)


# Объединяем order_products_train_df и products_df по product_id, чтобы получить product_name
mix_df = pd.merge(order_products_train_df, products_df[['product_id', 'product_name']], on='product_id')

# Объединяем с orders_df по order_id, чтобы получить order_id и product_name
trans_df = pd.merge(mix_df, orders_df[['order_id', 'order_dow']], on='order_id')

# Создаем транзакционную базу данных
# Группируем по order_id и собираем product_name в список
trans_data = trans_df.groupby('order_id')['product_name'].apply(lambda x: ', '.join(x)).reset_index()

# Находим транзакцию с наибольшим количеством товаров
trans_data['num_products'] = trans_data['product_name'].apply(lambda x: len(x.split(', ')))
max_trans = trans_data.loc[trans_data['num_products'].idxmax()]

# Выводим результат
print(f"Транзакция с наибольшим количеством товаров (order_id {max_trans['order_id']}):")
print(f"Количество товаров: {max_trans['num_products']}")
print(f"Список товаров: {max_trans['product_name']}")


Транзакция с наибольшим количеством товаров (order_id 2813632):
Количество товаров: 84
Список товаров: Asparagus, Organic Cucumber, Fresh Cauliflower, Cherubs Heavenly Salad Tomatoes, Extra Virgin Olive Oil, Basil Pesto, Mild Salsa, Garlic, Organic Large Extra Fancy Fuji Apple, Red Vine Tomato, Milano Milk Chocolate Cookies, Freshly Made. Filled with Creamy Ricotta, Aged Parmesan and Romano Cheeses Three Cheese Tortellini, Angel Hair Pasta, Meyer Lemon, Raisin Bran Cereal, Super Soft Taco Flour Tortillas, Less Sodium Soy Sauce, Original Rice Pilaf Mix, Baby Carrots, Asparation/Broccolini/Baby Broccoli, Macaroni Shells & White Cheddar Cheese, Sharp Cheddar Cheese, Unsweetened Vanilla Almond Milk, Parmesan Cheese, Slices Cheddar Cheese, Boneless Skinless Chicken Breast Fillets, Organic Whipped Naturally Buttery Spread, Caramel Cone Ice Cream, Dill Pickle Spears, Lightly Salted Baked Snap Pea Crisps, Rotini Pasta, Sensitive Toilet Paper, Dressing, Jersey Sweet Onion, Calorie Free, All Nat

###5.	Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета mlxtend. По бинарной базе данных определите и выведите на экран три наиболее популярных товара и определите количество покупок (транзакций) этих товаров.

In [ ]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
# Постройка транзакционной базы данных (замените `transaction_data` на ваш датафрейм)
transaction_list = trans_data['product_name'].apply(lambda x: x.split(', ')).tolist()

# Создание бинарной базы данных
trans = TransactionEncoder()
trans_list = trans.fit(transaction_list).transform(transaction_list)
df_binary = pd.DataFrame(trans_list, columns=trans.columns_)

# Определение количества покупок (транзакций) для каждого товара
# Мы суммируем количество 1-ц для каждого товара, чтобы найти количество транзакций, в которых он был куплен
counts = df_binary.sum().sort_values(ascending=False)

# Определение трех наиболее популярных товаров
items = counts.head(3)

# Вывод результатов
print("Три наиболее популярных товара и количество покупок:")
print(items)


Три наиболее популярных товара и количество покупок:
Banana                    18805
Bag of Organic Bananas    15480
Organic Strawberries      10894
dtype: int64


###6.	При помощи указанного в индивидуальном задании метода построения популярных наборов предметов постройте и выведите на экран популярный набор предметов с (абсолютной) минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии такого набора уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярного набора предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

In [ ]:
from mlxtend.frequent_patterns import fpgrowth
df_binary_sample = df_binary.sample(frac=0.5, random_state=1)
# Функция для нахождения частых наборов предметов с минимальной поддержкой
def frequent_item_sets(df, min_support):
    return fpgrowth(df, min_support=min_support, use_colnames=True)
# Попытка найти частые наборы с поддержкой не менее 3
try:
    frequent_items= frequent_item_sets(df_binary_sample, min_support=3/len(df_binary_sample))
    if frequent_items.empty:
        raise ValueError("Нет частых наборов с поддержкой не менее 3.")
except ValueError:
    # Попробовать с поддержкой не менее 2
    frequent_items = frequent_item_sets(df_binary_sample, min_support=2/len(df_binary_sample))

# Находим набор с максимальной длиной
if not frequent_items.empty:
    frequent_items['length'] = frequent_items['itemsets'].apply(lambda x: len(x))
    max_length_item_set = frequent_items.loc[frequent_items['length'].idxmax()]
    print("Популярный набор предметов с максимальной длиной и поддержкой:")
    print(max_length_item_set)
else:
    print("Не удалось найти частые наборы предметов с заданной поддержкой.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Популярный набор предметов с максимальной длиной и поддержкой:
support                                              0.000046
itemsets    (Ginger Cold Pressed Juice Beverage, Lemon Col...
length                                                     13
Name: 1171086, dtype: object


###7.	Используя пакет mlxtend или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов предметов с уровнем достоверности (confidence) не ниже.

In [ ]:
from mlxtend.frequent_patterns import fpgrowth, association_rules
# Постройка частых наборов предметов
frequent_items = fpgrowth(df_binary_sample, min_support=3/len(df_binary_sample), use_colnames=True)

# Генерация ассоциативных правил
min_confidence = 0.1  # Уровень достоверности
rules = association_rules(frequent_items, metric="confidence", min_threshold=min_confidence)

# Выводим результаты
print("Ассоциативные правила с уровнем достоверности не ниже {:.2f}:".format(min_confidence))
print(rules)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Ассоциативные правила с уровнем достоверности не ниже 0.10:
                                  antecedents  \
0                          (Organic Zucchini)   
1                          (Organic Zucchini)   
2                          (Organic Zucchini)   
3                          (Organic Zucchini)   
4                          (Organic Zucchini)   
...                                       ...   
6911565            (Pepperoni, Onions) Pizza)   
6911566        (Green Peppers, Onions) Pizza)   
6911567          (Onions) Pizza, Red Peppers)   
6911568  (Traditional Crust Supreme (Sausage)   
6911569                       (Onions) Pizza)   

                                               consequents  \
0                                 (Bag of Organic Bananas)   
1                                   (Organic Baby Spinach)   
2                                   (Organic Strawberries)   
3                                                 (Banana)   
4                                        

###8.	Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и выведите на экран ассоциативное правило, имеющее наилучшее значение этого показателя оценки.

In [ ]:
# Оптимизация поиска поддержки
support_dict = frequent_items.set_index('itemsets')['support'].to_dict()

# Рассчитайте рычаг для каждого правила
def cal_leverage(row, support_dict):
    antecedent_support = support_dict.get(frozenset(row['antecedents']), 0)
    consequent_support = support_dict.get(frozenset(row['consequents']), 0)
    return row['support'] - (antecedent_support * consequent_support)

# Применяем функцию для расчета рычага
rules['leverage'] = rules.apply(lambda row: cal_leverage(row, support_dict), axis=1)

# Найдите правило с наибольшим рычагом
if not rules.empty:
    best_rule = rules.loc[rules['leverage'].idxmax()]
    print("Ассоциативное правило с наибольшим рычагом:")
    print(best_rule)
else:
    print("Правила ассоциации не найдены.")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Ассоциативное правило с наибольшим рычагом:
antecedents           (Bag of Organic Bananas)
consequents             (Organic Strawberries)
antecedent support                    0.117935
consequent support                    0.083608
support                               0.023886
confidence                            0.202533
lift                                  2.422424
leverage                              0.014025
conviction                            1.149129
zhangs_metric                         0.665699
Name: 17951, dtype: object
